In [1]:
import pandas as pd
import numpy as np

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
df[['cough']].value_counts()

cough 
Mild      62
Strong    38
Name: count, dtype: int64

In [4]:
df[['gender']].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [5]:
df[['cough']].value_counts()

cough 
Mild      62
Strong    38
Name: count, dtype: int64

In [6]:
df[['city']].value_counts()

city     
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [7]:
# age -> nothing
# gender -> on hot encoding
# fever -> simple imuper for missing values 
# cough -> ordinal encoding
# city -> one hot encoding 


In [11]:
from sklearn.model_selection import train_test_split
x = df.drop(['has_covid'],axis=1)
y = df[['has_covid']]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

### manual way of transforming

In [12]:
# simple impiting
imputer = SimpleImputer()
x_train_fever = imputer.fit_transform(x_train[['fever']])
x_test_fever = imputer.fit_transform(x_test[['fever']])

In [19]:
ordE = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = ordE.fit_transform(x_train[['cough']])
x_test_cough = ordE.fit_transform(x_test[['cough']])


In [21]:

ohe = OneHotEncoder(dtype='int',sparse_output=False,drop='first')
x_train_gender_city =  ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])


In [23]:
x_train_transformed = np.concatenate([x_train_cough,x_train_fever,x_train_gender_city],axis=1)
x_test_transformed = np.concatenate([x_test_cough,x_test_fever,x_test_gender_city],axis=1)
x_train_transformed

array([[  0.        , 104.        ,   1.        ,   0.        ,
          0.        ,   1.        ],
       [  0.        , 104.        ,   1.        ,   0.        ,
          0.        ,   1.        ],
       [  0.        , 102.        ,   1.        ,   0.        ,
          0.        ,   1.        ],
       [  0.        ,  98.        ,   1.        ,   0.        ,
          1.        ,   0.        ],
       [  0.        , 103.        ,   0.        ,   0.        ,
          1.        ,   0.        ],
       [  1.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   1.        ],
       [  1.        , 101.        ,   1.        ,   0.        ,
          0.        ,   0.        ],
       [  1.        , 102.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  1.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ],
       [  1.        , 103.        ,   1.        ,   0.        ,
          1.        ,   0. 

### By ColumnTransformer

In [24]:
from sklearn.compose import ColumnTransformer

In [31]:
transformer = ColumnTransformer(transformers=[
    ('tf1',SimpleImputer(),['fever']),
    ('tf2',OrdinalEncoder(categories=[['Mild','Strong']]) , ['cough']),
    ('tf3',OneHotEncoder(dtype='int',sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [32]:
x_train_transformed = transformer.fit_transform(x_train)
x_test_transformed = transformer.transform(x_test)

In [33]:
x_train_transformed

array([[104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  42.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  44.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  74.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  24.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   5.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  24.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.    